In [ ]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import pyttsx3 as textspeech
import pandas as pd

engine = textspeech.init()

# Load the YOLO model for object detection
object_detection_model = YOLO("E:/zz/runs/runs/detect/train/weights/best.pt")

# Maintain a dictionary to track quantities of detected products
product_quantities = {}

# List to store detected products in the current frame
detected_products_in_frame = []

# CSV file to store attendance and billing results
csv_file_name = 'attendance_and_billing_results.csv'

# Create a pandas DataFrame to store the data
columns = ['Product', 'Time', 'Date', 'Quantity']
df = pd.DataFrame(columns=columns)

def mark_billing(product_name):
    global df

    time_now = datetime.now()
    t_str = time_now.strftime('%H:%M:%S')
    d_str = time_now.strftime('%d/%m/%Y')

    # Check if the product is already in the current frame
    if product_name in detected_products_in_frame:
        return

    # Check if the product is already in the DataFrame
    if product_name in df['Product'].values:
        # Increase the quantity for the existing product
        df.loc[df['Product'] == product_name, 'Quantity'] += 1
    else:
        # Add a new row for the product in the DataFrame
        new_row = {'Product': product_name, 'Time': t_str, 'Date': d_str, 'Quantity': 1}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    # Update product quantity in the dictionary
    if product_name in product_quantities:
        product_quantities[product_name] += 1
    else:
        product_quantities[product_name] = 1

    # Add the product to the list of detected products in the current frame
    detected_products_in_frame.append(product_name)

    # Write the DataFrame to the CSV file
    df.to_csv(csv_file_name, index=False)

    # Speak a statement about adding the product to the cart
    statement = f"{product_name} added to the cart"
    engine.say(statement)
    engine.runAndWait()

def detect_objects_and_billing(frame):
    global detected_products_in_frame
    results = object_detection_model(frame)[0]

    # Clear the list of detected products in the current frame
    detected_products_in_frame = []

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        product_name = object_detection_model.names[int(class_id)]

        mark_billing(product_name)

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, product_name, (int(x1) + 2, int(y2) - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2)

    return frame

def process_video():
    vid = cv2.VideoCapture(0)

    while True:
        success, frame = vid.read()
        if not success:
            break

        frame_objects = detect_objects_and_billing(frame.copy())

        cv2.imshow('Object Detection and Billing', frame_objects)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    process_video()


In [ ]:
import pandas as pd

def fetch_data_from_csv(csv_file_name):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_name)

        # Convert the DataFrame to a list of dictionaries
        data_list = df.to_dict(orient='records')

        # Transform the list of dictionaries to the desired format
        formatted_data = {row['Product']: row['Quantity'] for row in data_list}

        return formatted_data

    except FileNotFoundError:
        print(f"Error: The CSV file '{csv_file_name}' does not exist.")
        return None

# Example usage
csv_file_name = 'attendance_and_billing_results.csv'
data_from_csv = fetch_data_from_csv(csv_file_name)

if data_from_csv is not None:
    # Store products and quantities in a variable
    products_and_quantities = data_from_csv

    # Display the variable in dictionary format
    print(products_and_quantities)
